In [1]:
#Add Dependencies
import pandas as pd
import numpy as np
import psycopg2 as pg
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

### Connection to the Postgres database and Retrive data from the tables

In [2]:
#Read Data from the PostgressSql
conn = pg.connect(user = "postgres",
                                  password = "akshaj10",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "movie_data")
sql1 = "Select * From movie_metadata;"
sql2 = "Select * from award;"
movie_df = pd.read_sql_query(sql1, conn)
award_df = pd.read_sql_query(sql2, conn)
conn = None

In [3]:
#Load Data into Pandas Data frame and Display the dataframe.
movie_df.shape

(45006, 13)

In [4]:
#Load Data into Pandas Data frame and Display the dataframe.
award_df.shape

(10395, 8)

In [5]:
#Merge the two dataframe
combined_df = pd.merge(movie_df, award_df, left_on='title', right_on='film' )
combined_df.shape

(10477, 21)

In [6]:
combined_df.dtypes

imdb_id            object
budget             object
original_title     object
popularity         object
release_date       object
revenue           float64
runtime           float64
title              object
vote_average      float64
vote_count        float64
production         object
country            object
new_genre          object
index               int64
year_film           int64
year_ceremony       int64
ceremony            int64
category           object
name               object
film               object
winner               bool
dtype: object

In [7]:
# Filter all rows for US as the country
movie_us_df = combined_df[combined_df['country'] == 'United States of America'] 
movie_us_df.shape

(6740, 21)

In [8]:
# Find null values
for column in movie_us_df.columns:
    print(f"Column{column} has {movie_us_df[column].isnull().sum()} null values")

Columnimdb_id has 0 null values
Columnbudget has 0 null values
Columnoriginal_title has 0 null values
Columnpopularity has 0 null values
Columnrelease_date has 0 null values
Columnrevenue has 0 null values
Columnruntime has 0 null values
Columntitle has 0 null values
Columnvote_average has 0 null values
Columnvote_count has 0 null values
Columnproduction has 143 null values
Columncountry has 0 null values
Columnnew_genre has 0 null values
Columnindex has 0 null values
Columnyear_film has 0 null values
Columnyear_ceremony has 0 null values
Columnceremony has 0 null values
Columncategory has 0 null values
Columnname has 0 null values
Columnfilm has 0 null values
Columnwinner has 0 null values


In [9]:
#Set the index to IMDB_ID
movie_us_df = movie_us_df.set_index("imdb_id")

In [10]:
columns =['budget','original_title','popularity','revenue','runtime','vote_average','vote_count','production','country','new_genre','year_film', 'winner']

target = ['winner']

In [11]:
movie_us_df = movie_us_df.loc[:, columns].copy()
movie_us_df.shape


(6740, 12)

In [12]:
#Convert winner column into numbers 
movie_us_df["winner"] = movie_us_df["winner"].replace({True: 1, False: 0})
movie_us_df.shape

(6740, 12)

In [13]:
# Create a title Dataframe.
title_df=pd.DataFrame(data=movie_us_df['original_title'],index=movie_us_df.index,copy=True)
title_df.head()

,original_title
imdb_id,
tt0114709,Toy Story
tt0114709,Toy Story
tt0114709,Toy Story
tt0047437,Sabrina
tt0047437,Sabrina


In [14]:
# Remove the movie title from the dataframe
movie_us_df = movie_us_df.drop('original_title',1)
movie_us_df.head()

,budget,popularity,revenue,runtime,vote_average,vote_count,production,country,new_genre,year_film,winner
imdb_id,,,,,,,,,,,
tt0114709,30000000,21.946943,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995,0
tt0114709,30000000,21.946943,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995,0
tt0114709,30000000,21.946943,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995,0
tt0047437,2238813,7.3597410000000005,10000000.0,113.0,7.4,284.0,Paramount Pictures,United States of America,Comedy,1954,0
tt0047437,2238813,7.3597410000000005,10000000.0,113.0,7.4,284.0,Paramount Pictures,United States of America,Comedy,1954,0


In [16]:
movie_us_df['budget']=movie_us_df.budget.astype('int64')

In [17]:
movie_us_df.dtypes

budget            int64
popularity       object
revenue         float64
runtime         float64
vote_average    float64
vote_count      float64
production       object
country          object
new_genre        object
year_film         int64
winner            int64
dtype: object

In [18]:
# Generate our categorical variable list for movie dataset
movie_cat = movie_us_df.dtypes[movie_us_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
movie_us_df[movie_cat].nunique()

popularity    2390
production     527
country          1
new_genre       19
dtype: int64

In [19]:
# Drop null rows
movie_us_df = movie_us_df.dropna(axis = 0,how= 'any')
movie_us_df.shape


(6597, 11)

In [20]:
# Check how many labels each variable has

for col in movie_us_df.columns:
    print(col, ': ', len(movie_us_df[movie_cat].nunique()), ' labels')

budget :  4  labels
popularity :  4  labels
revenue :  4  labels
runtime :  4  labels
vote_average :  4  labels
vote_count :  4  labels
production :  4  labels
country :  4  labels
new_genre :  4  labels
year_film :  4  labels
winner :  4  labels


In [21]:
# let's examine how many columns we will obtain after one hot encoding these variables
encode_df=pd.get_dummies(movie_us_df[movie_cat], drop_first=True)
encode_df.shape

(6597, 2849)

In [22]:
# Merge one-hot encoded features and drop the originals
oscar_us_df = movie_us_df.merge(encode_df,left_index=True, right_index=True)
oscar_us_df = oscar_us_df.drop(movie_cat,1)
oscar_us_df.head(2)

,budget,revenue,runtime,vote_average,vote_count,year_film,winner,popularity_0.00118,popularity_0.001191,popularity_0.001224,...,new_genre_History,new_genre_Horror,new_genre_Music,new_genre_Mystery,new_genre_Romance,new_genre_Science Fiction,new_genre_TV Movie,new_genre_Thriller,new_genre_War,new_genre_Western
imdb_id,,,,,,,,,,,,,,,,,,,,,
tt0002101,0,0.0,88.0,4.0,1.0,1934,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0002101,0,0.0,88.0,4.0,1.0,1934,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Classification (RandomForestClassification)

In [23]:
# Define the features set.
X = oscar_us_df.copy()
X = X.drop('winner', axis=1)
X.shape

(36263, 2855)

In [24]:
# Define the target set(output label)
y = oscar_us_df['winner']
y.head(2)

imdb_id
tt0002101    0
tt0002101    0
Name: winner, dtype: int64

In [25]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [26]:
# Creating a StandardScaler instance.
scaler = StandardScaler()

In [27]:
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

In [28]:
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
# Create a random forest classifier.
#rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [30]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [31]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [32]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,6129,572
Actual 1,1426,939


In [33]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.7796161482461945

In [34]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,6129,572
Actual 1,1426,939


Accuracy Score : 0.7796161482461945
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.91      0.86      6701
           1       0.62      0.40      0.48      2365

    accuracy                           0.78      9066
   macro avg       0.72      0.66      0.67      9066
weighted avg       0.76      0.78      0.76      9066



In [35]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.04626111, 0.04801704, 0.06238899, ..., 0.00195893, 0.00351335,
       0.00152537])

In [36]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.09834461986702302, 'year_film'),
 (0.06599093057676561, 'vote_count'),
 (0.062388990400974495, 'runtime'),
 (0.05975625036057887, 'vote_average'),
 (0.048017038689409354, 'revenue'),
 (0.04626111028534994, 'budget'),
 (0.015766282899012923, 'popularity_12.90707'),
 (0.01477229877890347, 'popularity_10.36311'),
 (0.013640965661964079, 'popularity_11.51021'),
 (0.010774596265132122, 'popularity_8.389903'),
 (0.010527708325998971, 'popularity_14.319742999999999'),
 (0.010393018564397715, 'new_genre_Drama'),
 (0.007922596216072943, 'popularity_26.88907'),
 (0.00776225539982729, 'production_Twentieth Century Fox Film Corporation'),
 (0.007734661025244679, 'production_Paramount Pictures'),
 (0.006887522060404729, 'production_Metro-Goldwyn-Mayer (MGM)'),
 (0.006252891069172896, 'production_United Artists'),
 (0.00617217807356399, 'popularity_10.82887'),
 (0.005898235171899273, 'new_genre_Comedy'),
 (0.005414451346397395, 'new_genre_Crime'),
 (0.005313203686822805, 'popularity_12.677592'),